In [70]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [71]:
url = "https://www.sec.gov/cgi-bin/srch-edgar?text=FORM-TYPE=10-k+and+FILING-DATE=202102*"

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

rows = soup.findAll('tr')

new_rows = rows[12:93]

filing_date = []
company_name = []
form_url = []

form = []

wordList = []


print(len(rows))
print(len(new_rows))

for row in new_rows:
    data = row.findAll('td')

    filingDate = str(data[4]).split(">")[1].split('<')[0]

    filing_date.append(filingDate)


    for company in data[1:2]:
            results = company.findAll('a', href=True)
            companyName = str(results).split(">")[1].split('<')[0]

            #print(companyName)
            
            company_name.append(companyName)

    for link in data[2:3]:
        new_url = link.findAll('a', href=True)[1]

        new_url = "https://www.sec.gov" + str(new_url.get('href'))

        form_url.append(new_url)

#To scrape the second page for the 10-k link, then put the links into another list
for link in range(0, len(form_url)):
    url = str(form_url[link])

    HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

    response = requests.get(url, headers = HEADERS)

    soup = BeautifulSoup(response.text, 'html.parser')

    new_url = soup.findAll('a', href=True)[9]

    try:
        form.append("https://www.sec.gov" + str(new_url.get('href').split('=')[1]))
    except:
        form.append("https://www.sec.gov" + str(new_url.get('href')))

#Scrape the form url list for the words "supply chain" and dump to a list

for link in range(0, len(form)):
    url = str(form[link])

    HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

    response = requests.get(url, headers = HEADERS)

    soup = BeautifulSoup(response.text, 'html.parser')

    wordList.append(str(soup.findAll(text=re.compile("supply chain"))))

        


values = {'Company Name': company_name, 'Filing Date': filing_date, '10-k Links': form 'Text Data':wordList}

df = pd.DataFrame(values).to_csv('formdata.csv', mode='w', index=False)

print(len(form))
print(len(company_name))
print(len(wordList))
print(wordList)

94
81
81
81
4
['The extent of the impacts of the COVID-19 pandemic on our business and financial results will continue to depend on numerous evolving factors that we are not able to accurately predict and which will vary by market, including the duration and scope of the pandemic, global economic conditions during and after the pandemic, governmental actions that have been taken, or may be taken in the future, in response to the pandemic, and changes in customer behavior in response to the pandemic, some of which may be more than just temporary. Our global operations expose us to risks associated with the COVID-19 pandemic, which has continued to result in challenging operating environments. COVID-19 continues to spread across the globe to almost all of the countries and territories in which our products are developed, made, manufactured, distributed or sold. Authorities in many of these markets have implemented numerous measures to stall the spread and reduce the impact of COVID-19, i